# Colab Notebook for Creating & Pushing HuggingFace Datasets

In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [35]:
# a lot of these came from the evaluation script so there are some that are unnecessary
from datasets import load_dataset, get_dataset_config_names, Dataset
import random
import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import DataLoader
import pandas as pd
from datasets import Dataset
from sklearn.utils import resample
from sklearn.utils import resample
from copy import copy
from torch.utils.data import DataLoader
import argparse
from tqdm import tqdm

In [ ]:
SEED = 42
NUM_PROC=5
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
CACHE=None

In [ ]:
def fill_template(templates, values):
    temp = random.sample(templates,1)[0]
    for i in range(len(values)):
        #print(f"i: {i}, values: {values[i]}")
        temp = temp.replace("${"+str(i+1)+"}", values[i])
    return temp

def generate_eval(dataset_name, templates, split, label_column, choices, question=None, template_key=None, template_labels=None, generate_other=False, input_sentences=[]):
  """
    - dataset_name: Path name to HuggingFace repo
    - templates: 2D list of templates
    - split: list of dataset split we are going to use, i.e. ['split']
    - question: column label
    - choices: optional choices (choice1 vs choice2 that we embed)
    - template_key: 'question' for XCOPA, 'label' for XNLI
    - template_labels: ['cause', 'effect'] for XCOPA, [0, 1, 2] for XNLI
    - generate_other: needed for XNLI
    - input_sentences: needed for XStoryCloze
  """
  langs = get_dataset_config_names(dataset_name)
  # sanity check: remove any langs that have more than 2 letters (should only be 2 letter code)
  langs = [lang for lang in langs if len(lang) == 2]

  data = {}
  for lang in langs:
    print(f"loading dataset lang: {lang}")
    data[lang] = load_dataset(dataset_name, lang, split=split, cache_dir=CACHE)

  col_names = copy(data[langs[0]][0].column_names)
  col_names.remove(label_column)

  def create_statements_labels(example):
    template=""
    # XCOPA, XNLI
    if template_key:
      if template_labels:
        for idx, val in enumerate(template_labels):
          if example[template_key] == val:
            template = templates[idx]

    # XWinograd
    if not template:
      template=templates

    # Choose from templates given
    temp = random.choice(template)

    statements = []

    # for chosen template fill, and append to statements
    for i in range(len(choices)):
      values = []
      if question:
        values.append(example[question])
      if input_sentences:
        for ip in input_sentences:
          values.append(example[ip])
      values.append(example[choices[i]])
      statements.append(fill_template([temp], values))

    # XNLI, need to process
    if generate_other:
      # wrong answers
      other_indices = [i for i in range(3) if i != example[template_key]]
      xnli_dict = {i:"" for i in range(3)}
      xnli_dict[example[template_key]] = statements[0]

      for other_idx in other_indices:
        temp = random.choice(templates[other_idx])
        xnli_dict[other_idx] = fill_template([temp], [example[question], example['hypothesis']])

        # fetch sorted statements
        statements = [xnli_dict[i] for i in range(3)]

    # now actually make the changes to example['statement1']
    for i in range(len(statements)):
      example[f'statement{i+1}'] = statements[i]

    # if not XCOPA, XNLI, we have to adjust labels to match (0-indexed for argmax)
    if not template_key:
      example[label_column] = int(example[label_column]) - 1

    return example

  resulting_statements = {}
  for lang in langs:
    print(f"Processing {lang}...")
    resulting_statements[lang] = [split.map(create_statements_labels, remove_columns=col_names, num_proc=NUM_PROC) for split in data[lang]][0]

  return resulting_statements, langs

In [ ]:
def create_dataloaders(statements, langs):
  dataloader_dict = {}
  for lang in langs:
    dataloader_dict[lang] = DataLoader(statements[lang], batch_size=32, shuffle=False)
  return dataloader_dict

In [ ]:
# XCOPA
print("Processing XCOPA...")
dataset = "xcopa"
templates = [["The cause of \"${1}\" is that \"${2}\"", "\"${1}\" because \"${2}\"", "\"${1}\" due to \"${2}\""], ["The effect of \"${1}\" is that \"${2}\"", "\"${1}\" therefore \"${2}\"", "\"${1}\", so \"${2}\""]]
split = ['test']
label_column = 'label'
question = 'premise'
choices = ['choice1', 'choice2']
template_key = 'question'
template_labels = ['cause', 'effect']

xcopa_statements, xcopa_langs = generate_eval(dataset, templates, split, label_column, choices, question, template_key, template_labels)
xcopa_dataloaders = create_dataloaders(xcopa_statements, xcopa_langs)

Processing XCOPA...
loading dataset lang: et
loading dataset lang: ht
loading dataset lang: id
loading dataset lang: it
loading dataset lang: qu
loading dataset lang: sw
loading dataset lang: ta
loading dataset lang: th
loading dataset lang: tr
loading dataset lang: vi
loading dataset lang: zh
Processing et...


Map (num_proc=5):   0%|          | 0/500 [00:00<?, ? examples/s]

Processing ht...


Map (num_proc=5):   0%|          | 0/500 [00:00<?, ? examples/s]

Processing id...


...

Map (num_proc=5):   0%|          | 0/500 [00:00<?, ? examples/s]

Processing vi...


Map (num_proc=5):   0%|          | 0/500 [00:00<?, ? examples/s]

Processing zh...


Map (num_proc=5):   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
print(type(xcopa_statements['et']))

<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
print(xcopa_langs)

['et', 'ht', 'id', 'it', 'qu', 'sw', 'ta', 'th', 'tr', 'vi', 'zh']


In [ ]:
print(xcopa_statements['et'][0])

{'label': 0, 'statement1': '"Ese oli mullikilesse mässitud." due to "See oli õrn."', 'statement2': '"Ese oli mullikilesse mässitud." due to "See oli väike."'}


In [ ]:
def push_dataset(statements, langs, dataset_name):
  for lang_code in langs:
    statements[lang_code].push_to_hub(f"mbzuai-ugrip-statement-tuning/{dataset_name}", lang_code, split='test')

In [ ]:
# login to your cli notebook
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
push_dataset(xcopa_statements, xcopa_langs, 'xcopa')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/353 [00:00<?, ?B/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/3.25k [00:00<?, ?B/s]

In [ ]:
  # XNLI
print("Processing XNLI...")
dataset = "facebook/xnli"
# templates: 'Entailment', 'Neutral', 'Contradiction'
templates = [["\"${1}\" entails \"${2}\"", "\"${1}\"? yes, \"${2}\"", "Premise: \"${1}\", Hypothesis: \"${2}\", label: Entailment"],
            ["\"${1}\" is neutral with regards to \"${2}\"", "\${1}\? maybe, \"${2}\"", "Premise: \"${1}\", Hypothesis: \"${2}\", label: Neutral"],
            ["\"${1}\" contradicts \"${2}\"", "\"${1}\"? no, \"${2}\"", "Premise: \"${1}\", Hypothesis: \"${2}\", label: Contradiction"]]
split = ['test']
label_column = 'label'
question = 'premise'
choices = ['hypothesis']
# 0: Entailment, 1: Neutral, 2: Contradiction
template_key = 'label'
template_labels = [0, 1, 2]

xnli_statements, xnli_langs = generate_eval(dataset, templates, split, label_column, choices, question, template_key, template_labels, generate_other=True)
xnli_dataloaders = create_dataloaders(xnli_statements, xnli_langs)

Processing XNLI...


loading dataset lang: ar


Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5010 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2490 [00:00<?, ? examples/s]

...

Map (num_proc=5):   0%|          | 0/5010 [00:00<?, ? examples/s]

Processing vi...


Map (num_proc=5):   0%|          | 0/5010 [00:00<?, ? examples/s]

Processing zh...


Map (num_proc=5):   0%|          | 0/5010 [00:00<?, ? examples/s]

In [ ]:
print(xnli_statements['en'][0])

{'label': 2, 'statement1': '"Well, I wasn\'t even thinking about that, but I was so frustrated, and, I ended up talking to him again."? yes, "I havent spoken to him again."', 'statement2': '\\Well, I wasn\'t even thinking about that, but I was so frustrated, and, I ended up talking to him again.\\? maybe, "I havent spoken to him again."', 'statement3': 'Premise: "Well, I wasn\'t even thinking about that, but I was so frustrated, and, I ended up talking to him again.", Hypothesis: "I havent spoken to him again.", label: Contradiction'}


In [ ]:
push_dataset(xnli_statements, xnli_langs, 'xnli')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/505 [00:00<?, ?B/s]

...

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/6.19k [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/6.67k [00:00<?, ?B/s]

In [ ]:
# XWinograd
print("Processing XWinograd...")
dataset = "Muennighoff/xwinograd"
templates = ["In \"${1}\", _ is: \"${2}\"", "Q:\"${1}\", A: \"${2}\"", "The missing word in \"${1}\" is \"${2}\"", "_ in: \"${1}\" is \"${2}\"", "\"${1}\", _ is: \"${2}\""]
split = ['test']
label_column = 'answer'
question = 'sentence'
choices = ['option1', 'option2']

xwinograd_statements, xwinograd_langs = generate_eval(dataset, templates, split, label_column, choices, question)
xwinograd_dataloaders = create_dataloaders(xwinograd_statements, xwinograd_langs)

Processing XWinograd...
loading dataset lang: en


Generating test split:   0%|          | 0/2325 [00:00<?, ? examples/s]

loading dataset lang: fr


Generating test split:   0%|          | 0/83 [00:00<?, ? examples/s]

loading dataset lang: jp


...

Map (num_proc=5):   0%|          | 0/263 [00:00<?, ? examples/s]

Processing ru...


Map (num_proc=5):   0%|          | 0/315 [00:00<?, ? examples/s]

Processing zh...


Map (num_proc=5):   0%|          | 0/504 [00:00<?, ? examples/s]

In [ ]:
push_dataset(xwinograd_statements, xwinograd_langs, 'xwinograd')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/358 [00:00<?, ?B/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

In [ ]:
# XStoryCloze
print("Processing XStoryCloze...")
dataset = "juletxara/xstory_cloze"
templates = ["\"${1}\" \"${2}\" \"${3}\" \"${4}\" \"${5}\""]
split = ['eval']
label_column = 'answer_right_ending'
input_sentences=['input_sentence_1', 'input_sentence_2', 'input_sentence_3', 'input_sentence_4']
choices=['sentence_quiz1', 'sentence_quiz2']

xstorycloze_statements, xstorycloze_langs = generate_eval(dataset, templates, split, label_column, choices, input_sentences=input_sentences)
xstorycloze_dataloaders = create_dataloaders(xstorycloze_statements, xstorycloze_langs)

Processing XStoryCloze...
loading dataset lang: ar


Generating train split:   0%|          | 0/360 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/1511 [00:00<?, ? examples/s]

loading dataset lang: en


Generating train split:   0%|          | 0/360 [00:00<?, ? examples/s]

...

Map (num_proc=5):   0%|          | 0/1511 [00:00<?, ? examples/s]

Processing te...


Map (num_proc=5):   0%|          | 0/1511 [00:00<?, ? examples/s]

Processing zh...


Map (num_proc=5):   0%|          | 0/1511 [00:00<?, ? examples/s]

In [ ]:
push_dataset(xstorycloze_statements, xstorycloze_langs, 'xstorycloze')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/373 [00:00<?, ?B/s]

...

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/3.10k [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/3.44k [00:00<?, ?B/s]